# ShapeNetSEM Subset Creation

The ShapeNet SEM dataset is very large with roughly 12k meshes, this is a notebook to create a smaller subset of the dataset which can then be used to play around with and to create a proof of concept model.

## 1. Creating a subset of the dataset

In [30]:
import pandas as pd
import random

In [5]:
metadata_df = pd.read_csv('../Data/ShapeNetSem/Files/metadata.csv')
categories_df = pd.read_csv('../Data/ShapeNetSem/Files/categories.synset.csv')

In [10]:
metadata_df.head()

,fullId,category,wnsynset,wnlemmas,up,front,unit,aligned.dims,isContainerLike,surfaceVolume,solidVolume,supportSurfaceArea,weight,staticFrictionForce,name,tags
0,wss.1004f30be305f33d28a1548e344f0e2e,WallArt,n3445436,globe,"0\,0\,1","0\,-1\,0",0.015032,"111.97104\,84.16881\,0.0",NaN,NaN,NaN,NaN,NaN,NaN,globe blue spruce,*
1,wss.100f39dce7690f59efb94709f30ce0d2,"Chair,Recliner",n4069540,"recliner,reclining chair,lounger","0\,0\,1","0\,-1\,0",0.012947,"111.34567\,100.547745\,96.13275",NaN,NaN,NaN,NaN,NaN,NaN,couch,carpet recliner
2,wss.101354f9d8dede686f7b08d9de913afe,"Speaker,_Attributes",n3696785,"loudspeaker,speaker,speaker unit,loudspeaker s...",NaN,NaN,0.013620,"43.43313\,60.591843\,32.17259",NaN,NaN,NaN,NaN,NaN,NaN,sound speaker,"audio,audio speaker,music,sound,sound speaker,..."
3,wss.1018f01d42ae7fad52249d8432f6087e,Sword,n4380981,"sword,blade,brand,steel",NaN,NaN,0.010424,"78.23693\,4.360932\,18.058533",NaN,NaN,NaN,NaN,NaN,NaN,sword,"blade,sword,weapon"
4,wss.1022fe7dd03f6a4d4d5ad9f13ac9f4e7,"Chair,OfficeChair",n3005231,chair,"0\,0\,1","0\,1\,0",0.017984,"60.366123\,98.00925\,66.79712",NaN,NaN,NaN,NaN,NaN,NaN,office chair,"fauteuil de bureau,office chair"


In [11]:
categories_df.head()

,category,matchLevel,synset,synset words,synset gloss
0,1Shelves,0,n4197095,shelf,a support that consists of a horizontal surfac...
1,2Shelves,0,n4197095,shelf,a support that consists of a horizontal surfac...
2,3Shelves,0,n4197095,shelf,a support that consists of a horizontal surfac...
3,4Shelves,0,n4197095,shelf,a support that consists of a horizontal surfac...
4,5Shelves,0,n4197095,shelf,a support that consists of a horizontal surfac...


In [17]:
num_cat_sample = 10
meshes_per_cat = 10

sampled_categories = categories_df.sample(n=num_cat_sample, random_state=42)
sampled_metadata_df = pd.DataFrame()
print(type(sampled_categories))
for category in sampled_categories['category']:
    print(category)

"""
final_df = pd.merge(sampled_metadata_df, categories_df[['category', 'synset words', 'synset gloss']], on='category', how='left')
final_df = final_df[['fullId', 'category', 'name', 'tags', 'synset words', 'synset gloss']]
final_df.head()"""

<class 'pandas.core.frame.DataFrame'>
AccentChair
Tank
Ladder
Rabbit
ComputerMouse
Wallet
Vanity
Motorcycle
Butterfly
Ottoman


"\nfinal_df = pd.merge(sampled_metadata_df, categories_df[['category', 'synset words', 'synset gloss']], on='category', how='left')\nfinal_df = final_df[['fullId', 'category', 'name', 'tags', 'synset words', 'synset gloss']]\nfinal_df.head()"

In [29]:
num_cat_sample = 20
meshes_per_cat = 10

sampled_categories = categories_df.sample(n=num_cat_sample, random_state=42)
sampled_metadata_df = pd.DataFrame()

for category in sampled_categories['category']:
    try:
        category_metadata = metadata_df[metadata_df['category'].str.contains(category, case=False, na=False)]
        sample_size = min(meshes_per_cat, len(category_metadata))
        sampled_category_metadata = category_metadata.sample(n=sample_size, random_state=42)
        sampled_metadata_df = pd.concat([sampled_metadata_df, sampled_category_metadata])
        if sample_size < meshes_per_cat:
            print(category, sample_size)
    except ValueError:
        print(category)
        continue

final_df = pd.merge(sampled_metadata_df, categories_df[['category', 'synset words', 'synset gloss']], on='category', how='left')
final_df = final_df[['fullId', 'category', 'name', 'tags', 'synset words', 'synset gloss']]
final_df.shape

Ladder 4
Rabbit 0
Wallet 1
Vanity 4
Motorcycle 7
Butterfly 1
Pedestal 4


(151, 6)

In [36]:
def create_subset_by_random_sampling_category(num_cat_sample = 20, meshes_per_cat = 10):
    remaining_categories = categories_df['category'].to_list()
    sampled_categories = []
    sampled_metadata_df = pd.DataFrame()

    while len(sampled_categories) < num_cat_sample:
        sampled_category = random.choice(remaining_categories)
        category_metadata = metadata_df[metadata_df['category'].str.contains(sampled_category, case=False, na=False)]
        num_samples = len(category_metadata)
        if num_samples >= meshes_per_cat:
            sample_size = min(meshes_per_cat, len(category_metadata))
            sampled_metadata = category_metadata.sample(n=sample_size, random_state=42)
            sampled_metadata_df = pd.concat([sampled_metadata_df, sampled_metadata])
            sampled_categories.append(sampled_category)
        
        remaining_categories.remove(sampled_category)

    final_df = pd.merge(sampled_metadata_df, categories_df[['category', 'synset words', 'synset gloss']], on='category', how='left')
    final_df = final_df[['fullId', 'category', 'name', 'tags', 'synset words', 'synset gloss']]
    return final_df

ssm_subset = create_subset_by_random_sampling_category(num_cat_sample=20, meshes_per_cat=10)
ssm_subset

,fullId,category,name,tags,synset words,synset gloss
0,wss.7735d2b98817c8061541581d0732faf4,"ChestOfDrawers,Armoire,Wardrobe",wardrobe,NaN,NaN,NaN
1,wss.c0940c5f37974c7af46d6fc43de3fc46,"ChestOfDrawers,Armoire,Wardrobe,_PilotStudyModels",wardrobe,"bedroom,furniture,wardrobe",NaN,NaN
2,wss.13dbeeacdabce3b694658a0201ba0367,"ChestOfDrawers,Armoire,Dresser,Wardrobe",armoire wardrobe,"architecture,armoire,dresser,furniture,traditi...",NaN,NaN
3,wss.cb48ec828b688a78d747fd5e045d7490,"ChestOfDrawers,Armoire,Wardrobe",wardrobe,"vaatekaappi,wardrobe",NaN,NaN
4,wss.4fe4ece12ccf5fc787c4d5470f8b581,"ChestOfDrawers,Armoire,Wardrobe",ikea aneboda wardrobe,"aneboda,birch,closet,foil,ikea,plastic,storage...",NaN,NaN
...,...,...,...,...,...,...
195,wss.a1a6eaea12fdbee9a631c7cd3764c770,"Lamp,CeilingLamp",chalen pendant lamp fixture,"lamp,light,luminaire,pendant",NaN,NaN
196,wss.297d929269bb62da43fdcbcacbbed64c,"Lamp,CeilingLamp",stained glass hanging lamp,"architecture,ceiling,details,glass,hanging,int...",NaN,NaN
197,wss.3b3fdf5af4afca7e35836c728d324152,"Lamp,CeilingLamp",kare hanging lamp beach h auml ngeleuch...,"decoration,design,hanging lamp,kare,leuchte,li...",NaN,NaN
198,wss.1dbb34b55a78c1aea72fb89aa30b6583,"Lamp,CeilingLamp",light,*,NaN,NaN


In [67]:
def create_subset_by_random_sampling(metadata_df, categories_df, num_cat_sample = 20, meshes_per_cat = 10):
    remaining_categories = categories_df['category'].to_list()
    sampled_categories = []
    sampled_metadata_df = pd.DataFrame()
    expanded_metadata_df = pd.DataFrame()

    while len(sampled_categories) < num_cat_sample:
        sampled_category = random.choice(remaining_categories) # select a category at random
        category_metadata = metadata_df[metadata_df['category'].str.contains(sampled_category, case=False, na=False)] # for cases where there might be more than one category
        num_samples = len(category_metadata)
        if num_samples >= meshes_per_cat: # make sure there are minimum meshes and only sample in that case
            sample_size = min(meshes_per_cat, len(category_metadata))
            sampled_metadata = category_metadata.sample(n=sample_size, random_state=42)
            sampled_metadata_df = pd.concat([sampled_metadata_df, sampled_metadata])
            sampled_categories.append(sampled_category)
        
        remaining_categories.remove(sampled_category)

    for _, row in sampled_metadata_df.iterrows():
        # Split the categories by commas (for cases like 'Laptop, PC')
        categories_list = row['category'].split(',')
        print(categories_list)
        # For each category in the list, merge the relevant synset information
        synset_words = []
        synset_gloss = []
        for category in categories_list:
            if '_' not in category: # To remove any non category values like _Attribute
                category = category.strip()  # Remove any leading/trailing spaces
                category_info = categories_df[categories_df['category'].str.contains(category, case=False, na=False)] # Find category info
                synset_words.extend(category_info['synset words'].to_list()) # Add info to lists
                synset_gloss.extend(category_info['synset gloss'].to_list()) # Add info to lists

        if not category_info.empty:
            # Create a new row with the current metadata and synset info
            expanded_row = row.copy()  # Copy the current row
            expanded_row['synset words'] = synset_words # Use extended list of all sub categories 
            expanded_row['synset gloss'] = synset_gloss # Use extended list of all sub categories
            
            # Append the expanded row to the new DataFrame
            expanded_metadata_df = pd.concat([expanded_metadata_df, expanded_row.to_frame().T], ignore_index=True)

    final_df = expanded_metadata_df[['fullId', 'category', 'name', 'tags', 'synset words', 'synset gloss']]
    # final_df.to_csv('subset_test.csv')
    return final_df

ssm_subset = create_subset_by_random_sampling(metadata_df, categories_df, num_cat_sample=20, meshes_per_cat=10)
ssm_subset.to_csv('ssm_subset.csv', index=False)

['Basket']
['_StanfordSceneDBModels', 'Basket']
['Basket']
['_StanfordSceneDBModels', 'Basket']
['Basket']
['_StanfordSceneDBModels', 'Basket']
['Basket']
['_StanfordSceneDBModels', 'Basket']
['_StanfordSceneDBModels', 'Basket']
['_PilotStudyModels', '_StanfordSceneDBModels', '_SceneGalleryModels', 'Basket']
['Stool', 'Barstool', '_AttributesTrain']
['Stool', 'Barstool', '_StanfordSceneDBModels', '_AttributesTrain']
['Stool', 'Barstool', '_AttributesTrain']
['Stool', 'Barstool']
['Stool', 'Barstool', '_AttributesTrain']
['Stool', 'Barstool', '_AttributesTrain']
['Stool', 'Barstool', '_AttributesTrain']
['Stool', 'Barstool', '_AttributesTrain']
['Stool', 'Barstool']
['Stool', 'Barstool', '_AttributesTrain']
['Chair', 'OfficeChair']
['Chair', 'OfficeChair']
['Chair', 'OfficeChair']
['Chair', 'OfficeChair']
['Chair', 'OfficeChair']
['Chair', 'OfficeChair', '_PilotStudyModels']
['Chair', 'OfficeChair']
['Chair', 'OfficeChair']
['Chair', 'OfficeChair', '_PilotStudyModels', '_StanfordSceneDB

## 2. Creating a semantic descriptor (text prompt)

In [68]:
import pandas as pd
import random

In [69]:
df = pd.read_csv('ssm_subset.csv')
df.head()

,fullId,category,name,tags,synset words,synset gloss
0,wss.121cf1ae08740534b8183a4a81361b94,Basket,basket,*,"['basket,handbasket']",['a container that is usually woven and has ha...
1,wss.28c7e4f134a6a30aa23949c21eddef76,"_StanfordSceneDBModels,Basket",kailey stewart recycling bin,NaN,"['basket,handbasket']",['a container that is usually woven and has ha...
2,wss.3c06fb3595c39a4eeb6c29873b08c02,Basket,laundry basket with wheel,"basket,laundry,sepet,tekerlekli","['basket,handbasket']",['a container that is usually woven and has ha...
3,wss.1444822d28a7f2632a526e2e9a7e9ae,"_StanfordSceneDBModels,Basket",clothes hamper,"clothes,hamper","['basket,handbasket']",['a container that is usually woven and has ha...
4,wss.2d8bb293b226dcb678c34bb504731cb9,Basket,basket,picnic basket,"['basket,handbasket']",['a container that is usually woven and has ha...


We need to create a semantic natural language descriptor using name, tags, synset words and synset_gloss. We will also need some text variation while doing this to improve generalisation. To perform this task, we will have two approaches to create different types of semantic descriptors that will be used as the text prompt for our multimodal model, these are: -
1. Template Approach: using a template where we will simply substitute values of the columns
2. LLM Appproach: giving the values along with a prompt to an LLM and getting a generated natural language description

### 2.1 Template Descriptor

Eg and variations: 

- **Template:** "A [name] which is commonly known as [tags]. It is associated with the following characteristics: [synset words]. A general description of this item is: [synset gloss]." 
- **Output:** "A Wooden Chair, which is commonly known as furniture, chair, seating. It is associated with the following characteristics: chair, seating, armchair. A general description of this item is: A piece of furniture used for sitting."

- **Template:**"The [name] is a [synset words] often used for [tags]. It is made of [material] and can be described as: [synset gloss]." 
- **Output:**  "The Wooden Chair is a chair, seating, armchair often used for furniture, seating. It is made of wood and can be described as: A piece of furniture used for sitting."

- **Template:**"[name] is a [synset words] designed for [tags]. It is a [material] item that serves the purpose of [synset gloss]." 
- **Output:**  "Wooden Chair is a chair, seating designed for furniture, seating. It is a wooden item that serves the purpose of: A piece of furniture used for sitting."


Example: For a Chair:

* Name: "Wooden Chair"
* Tags: "furniture, chair, seating"
* Synset Words: "chair, seating, armchair"
* Synset Gloss: "A piece of furniture used for sitting."
The generated description could be: "A Wooden Chair, which is commonly known as furniture, chair, seating. It is associated with the following characteristics: chair, seating, armchair. A general description of this item is: A piece of furniture used for sitting."

### 2.2 LLM Generated Descriptor

Given below is an example prompt to the LLM:

[prompt start]

You are an expert in 3D object recognition. Given the following attributes of an object, generate a detailed and natural language description:

- Name: {name}
- Tags: {tags}
- Synset Words: {synset_words}
- Synset Gloss: {synset_gloss}

Based on these, describe the object as if explaining it to someone who has never seen it before.

[prompt end]

Such an approach could result in a more natural language descriptor that goes beyond the listed values.